# Policy for Amazon Bedrock AgentCore - エンドツーエンド チュートリアル

## 概要

このノートブックでは、**Policy for Amazon Bedrock AgentCore** の実装方法を示します。これは、Cedar ポリシーと JWT トークンクレームを使用して AI エージェントのツール呼び出しに対するきめ細かいアクセス制御を可能にし、Amazon Bedrock AgentCore Gateway で適用する機能です。

### 学習内容

- Amazon Cognito を設定して JWT トークンにカスタムクレームを追加する方法
- Amazon Bedrock AgentCore Identity 経由で提供される JWT クレームを検証する Cedar ポリシーの作成方法
- 属性ベースアクセス制御（ABAC）パターンの実装方法
- 接続された Amazon Bedrock AgentCore Gateway で異なるクレームシナリオを使用してポリシー適用をテストおよび検証する方法

### 主要な概念

**Policy for Amazon Bedrock AgentCore**: JWT トークンクレームをプリンシパル属性として使用し、きめ細かいポリシーに対してアクセスリクエストを評価する Cedar ベースのポリシーエンジン。

**Amazon Bedrock AgentCore Identity**: Amazon Cognito または OAuth をサポートする他の Identity Provider（IdP）と統合し、リクエストを認証してポリシー評価用の JWT クレームを抽出します。

**Amazon Bedrock AgentCore Gateway**: MCP を使用してツールを大規模に構築、デプロイ、検出、接続するための、開発者にとって簡単で安全な方法。

**[Cedar Policies](https://www.cedarpolicy.com/en)**: 誰（プリンシパル）がどのリソースに対してどのアクション（アクション）を実行できるかを、オプションの条件付きで定義する宣言型ポリシー。詳細は [cedarpolicy.com](https://www.cedarpolicy.com/en) をご覧ください。

### サンプルアーキテクチャ

```
                                ┌───────────────────────┐
                                │  Policy for AgentCore │
                                │  (Cedar Policies)     │
                                │                       │
                                │  評価対象:            │
                                │  - principal tags     │
                                │  - context.input      │
                                │  - resource           │
                                └───────────┬───────────┘
                                            │ attached
                                            ▼
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────┐
│   Amazon        │  JWT Token  │  Amazon Bedrock       │             │   Lambda    │
│   Cognito       │────────────▶│  AgentCore Gateway    │────────────▶│   Target    │
│   + AWS Lambda  │  with       │                       │  if ALLOWED │   (Tool)    │
└─────────────────┘  claims     └───────────────────────┘             └─────────────┘
```

### 前提条件

- 適切な IAM 権限を持つ AWS アカウント
- OAuth オーソライザーで設定済みの Amazon Bedrock AgentCore Gateway
- アプリクライアント（M2M）を持つ Amazon Cognito User Pool
- boto3 と requests がインストールされた Python 3.8+
- AWS 認証情報が設定済み

---

## パート 1: セットアップと設定

まず、必要な依存関係をインストールし、設定を初期化しましょう。

In [ ]:
# Install required packages
%pip install -r requirements.txt

In [ ]:
import json
import os
import time
import base64
import zipfile
import tempfile
from pathlib import Path
from typing import Dict, Any, Optional, List

import boto3
import requests
from botocore.exceptions import ClientError

print("✓ ライブラリのインポートに成功しました")

### ステップ 1.1: 設定の読み込みまたは作成

このチュートリアルには、Amazon Bedrock AgentCore Gateway の詳細と Amazon Cognito クライアント情報を含む `gateway_config.json` ファイルが必要です。これらのリソースがまだない場合は、同じフォルダに用意されているユーティリティスクリプト `setup-gateway.py` を実行して作成できます。

#### 期待される設定構造

```json
{
  "gateway_url": "https://<gateway-id>.gateway.policy-registry.<region>.amazonaws.com/mcp",
  "gateway_id": "<gateway-id>",
  "gateway_arn": "arn:aws:policy-registry:<region>:<account-id>:gateway/<gateway-id>",
  "region": "<region>",
  "client_info": {
    "client_id": "<cognito-app-client-id>",
    "client_secret": "<cognito-app-client-secret>",
    "user_pool_id": "<region>_<pool-id>",
    "token_endpoint": "https://<domain>.auth.<region>.amazoncognito.com/oauth2/token"
  },
  "policy_engine_id": "<optional-policy-engine-id>"
}
```

#### 前提条件のセットアップ

これらのリソースがまだ設定されていない場合は、このノートブックから直接セットアップスクリプトを実行できます。スクリプトは以下を行います：
- OAuth 認可を持つ Amazon Bedrock AgentCore Gateway を作成
- テスト用のサンプル Refund Lambda 関数を作成
- Lambda を Gateway のターゲットとして接続
- 設定を `gateway_config.json` に保存

以下のセルを実行してセットアップスクリプトを実行します：

In [ ]:
# Configure the region for the setup (change as needed)
SETUP_REGION = "us-east-1"  # Change to your preferred region

# Optional: Specify an IAM role ARN if you have one with the trust relationship configured
# Leave as None to create a new role automatically
SETUP_ROLE_ARN = None  # e.g., "arn:aws:iam::123456789012:role/MyGatewayRole"

In [ ]:
# Run this cell to set up the Gateway and Lambda target automatically
import subprocess
import sys

cmd = [sys.executable, "setup-gateway.py", "--region", SETUP_REGION]
if SETUP_ROLE_ARN:
    cmd.extend(["--role-arn", SETUP_ROLE_ARN])

result = subprocess.run(cmd, capture_output=False, text=True)
if result.returncode != 0:
    print(f"セットアップ失敗、リターンコード: {result.returncode}")

また、以下のガイドに従って手動でリソースをセットアップすることもできます：

1. **Amazon Bedrock AgentCore Gateway**: [Gateway クイックスタートガイド](https://docs.aws.amazon.com/policy-registry/latest/devguide/gateway.html)
2. **Gateway へのターゲットの追加**: [Gateway ターゲットドキュメント](https://docs.aws.amazon.com/policy-registry/latest/devguide/gateway-targets.html)
3. **Amazon Cognito User Pool**: [Amazon Cognito デベロッパーガイド](https://docs.aws.amazon.com/cognito/latest/developerguide/cognito-user-pools.html)

In [ ]:
# Template for gateway_config.json
CONFIG_TEMPLATE = {
    "gateway_url": "https://<gateway-id>.gateway.policy-registry.<region>.amazonaws.com/mcp",
    "gateway_id": "<gateway-id>",
    "gateway_arn": "arn:aws:policy-registry:<region>:<account-id>:gateway/<gateway-id>",
    "region": "<region>",
    "client_info": {
        "client_id": "<cognito-app-client-id>",
        "client_secret": "<cognito-app-client-secret>",
        "user_pool_id": "<region>_<pool-id>",
        "token_endpoint": "https://<domain>.auth.<region>.amazoncognito.com/oauth2/token",
    },
}


def load_or_create_gateway_config() -> Dict[str, Any]:
    """
    Load gateway configuration from gateway_config.json.
    If the file doesn't exist, create a template for the user to fill in.
    """
    config_path = Path.cwd() / "gateway_config.json"

    if not config_path.exists():
        # Create template file
        with open(config_path, "w", encoding="utf-8") as f:
            json.dump(CONFIG_TEMPLATE, f, indent=2)

        print("⚠️  gateway_config.json not found!")
        print(f"\n✓ Created template at: {config_path}")
        print("\nPlease fill in the configuration with your actual values:")
        print("  1. Set up an Amazon Bedrock AgentCore Gateway")
        print("  2. Create an Amazon Cognito User Pool with an app client (M2M)")
        print("  3. Update gateway_config.json with your resource details")
        print("  4. Re-run this cell")
        print("\nDocumentation:")
        print(
            "  - Gateway: https://docs.aws.amazon.com/policy-registry/latest/devguide/gateway.html"
        )
        print(
            "  - Cognito: https://docs.aws.amazon.com/cognito/latest/developerguide/cognito-user-pools.html"
        )
        raise FileNotFoundError(
            "Please configure gateway_config.json and re-run this cell."
        )

    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    # Validate required fields
    required_fields = [
        "gateway_url",
        "gateway_id",
        "gateway_arn",
        "region",
        "client_info",
    ]
    missing = [
        f for f in required_fields if f not in config or "<" in str(config.get(f, ""))
    ]

    if missing:
        print("⚠️  Configuration incomplete!")
        print(f"   Please update these fields in gateway_config.json: {missing}")
        raise ValueError(f"Missing or placeholder values in config: {missing}")

    # Validate client_info fields
    client_info_fields = [
        "client_id",
        "client_secret",
        "user_pool_id",
        "token_endpoint",
    ]
    client_info = config.get("client_info", {})
    missing_client = [
        f
        for f in client_info_fields
        if f not in client_info or "<" in str(client_info.get(f, ""))
    ]

    if missing_client:
        print("⚠️  Client info incomplete!")
        print(f"   Please update client_info fields: {missing_client}")
        raise ValueError(
            f"Missing or placeholder values in client_info: {missing_client}"
        )

    return config


# Load configuration
CONFIG = load_or_create_gateway_config()

# Extract key values
REGION = CONFIG["region"]
GATEWAY_URL = CONFIG["gateway_url"]
GATEWAY_ID = CONFIG["gateway_id"]
GATEWAY_ARN = CONFIG["gateway_arn"]
USER_POOL_ID = CONFIG["client_info"]["user_pool_id"]
CLIENT_ID = CONFIG["client_info"]["client_id"]
CLIENT_SECRET = CONFIG["client_info"]["client_secret"]
TOKEN_ENDPOINT = CONFIG["client_info"]["token_endpoint"]
POLICY_ENGINE_ID = CONFIG.get("policy_engine_id")

print("✓ Configuration loaded successfully")
print(f"  Region: {REGION}")
print(f"  Gateway ID: {GATEWAY_ID}")
print(f"  Gateway URL: {GATEWAY_URL}")
print(f"  User Pool ID: {USER_POOL_ID}")
print(f"  Policy Engine ID: {POLICY_ENGINE_ID or 'Not configured yet'}")

### ステップ 1.2: AWS クライアントの初期化

このチュートリアルに必要な boto3 クライアントを作成します。

In [ ]:
# AWS クライアントを初期化
session = boto3.Session(region_name=REGION)

lambda_client = session.client("lambda")
cognito_client = session.client("cognito-idp")
iam_client = session.client("iam")
sts_client = session.client("sts")

# Policy Engine と Cedar ポリシー管理用の AgentCore Control クライアント
policy_client = session.client("bedrock-agentcore-control", region_name=REGION)

# 現在のアカウント情報を取得
ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS クライアントを初期化しました")
print(f"  アカウント ID: {ACCOUNT_ID}")
print(f"  リージョン: {REGION}")

### ステップ 1.3: Gateway オーソライザー設定の検証

このステップでは、Gateway の JWT オーソライザーが Cognito アクセストークン用に正しく設定されていることを確認します。

**重要**: Amazon Cognito アクセストークンには `aud`（オーディエンス）クレームが含まれていません。Gateway に `allowedAudience` が設定されている場合、トークン検証は 401 エラーで失敗します。このステップでは設定を確認し、必要に応じて修正します。

In [ ]:
# Gateway control client for managing gateway configuration
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)


def get_gateway_details() -> Dict[str, Any]:
    """Get current gateway details."""
    return gateway_control_client.get_gateway(gatewayIdentifier=GATEWAY_ID)


def wait_for_gateway_ready(max_wait: int = 300, poll_interval: int = 5) -> bool:
    """Wait for gateway to reach READY state."""
    terminal_states = {"READY", "FAILED", "UPDATE_UNSUCCESSFUL"}
    start_time = time.time()

    while time.time() - start_time < max_wait:
        gateway = get_gateway_details()
        status = gateway.get("status", "UNKNOWN")
        print(f"  Gateway status: {status}")

        if status == "READY":
            return True
        if status in terminal_states:
            print(f"  ✗ Gateway reached terminal state: {status}")
            return False

        time.sleep(poll_interval)

    print("  ✗ Timeout waiting for gateway")
    return False


def validate_and_fix_gateway_authorizer() -> bool:
    """
    Validate gateway authorizer configuration and fix if needed.

    Cognito access tokens don't have an 'aud' claim, so allowedAudience
    must not be set or the gateway will reject valid tokens.

    Returns:
        True if configuration is valid or was fixed successfully
    """
    print("\nValidating Gateway Authorizer Configuration")
    print("=" * 70)

    gw = get_gateway_details()
    jwt_config = gw.get("authorizerConfiguration", {}).get("customJWTAuthorizer", {})

    # Check current configuration
    discovery_url = jwt_config.get("discoveryUrl")
    allowed_clients = jwt_config.get("allowedClients", [])
    allowed_audience = jwt_config.get("allowedAudience", [])
    allowed_scopes = jwt_config.get("allowedScopes", [])

    print(f"  Discovery URL: {discovery_url or 'NOT SET'}")
    print(f"  Allowed Clients: {allowed_clients}")
    print(f"  Allowed Audience: {allowed_audience}")
    print(f"  Allowed Scopes: {allowed_scopes}")

    # Build expected discovery URL
    expected_discovery_url = f"https://cognito-idp.{REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"

    # Check if configuration needs fixing
    needs_fix = False
    reasons = []

    if not discovery_url:
        needs_fix = True
        reasons.append("Discovery URL not set")
    elif discovery_url != expected_discovery_url:
        needs_fix = True
        reasons.append("Discovery URL mismatch")

    if CLIENT_ID not in allowed_clients:
        needs_fix = True
        reasons.append(f"Client ID {CLIENT_ID} not in allowed clients")

    # Cognito access tokens don't have 'aud' claim - allowedAudience must be empty
    if allowed_audience:
        needs_fix = True
        reasons.append(
            "allowedAudience is set but Cognito access tokens don't have 'aud' claim"
        )

    if not needs_fix:
        print("\n✓ Gateway authorizer configuration is valid")
        return True

    print("\n⚠️  Configuration needs fixing:")
    for reason in reasons:
        print(f"   - {reason}")

    # Fix the configuration
    print("\n⏳ Updating gateway authorizer configuration...")

    # Get scope from config if available
    scope = CONFIG.get("client_info", {}).get("scope", "")

    new_auth_config = {
        "customJWTAuthorizer": {
            "discoveryUrl": expected_discovery_url,
            "allowedClients": [CLIENT_ID],
            # Do NOT set allowedAudience - Cognito access tokens don't have 'aud' claim
        }
    }

    # Add scope if configured
    if scope:
        new_auth_config["customJWTAuthorizer"]["allowedScopes"] = [scope]

    try:
        gateway_control_client.update_gateway(
            gatewayIdentifier=GATEWAY_ID,
            name=gw.get("name"),
            roleArn=gw.get("roleArn"),
            protocolType=gw.get("protocolType", "MCP"),
            authorizerType="CUSTOM_JWT",
            authorizerConfiguration=new_auth_config,
            policyEngineConfiguration=gw.get("policyEngineConfiguration", {}),
        )

        print("\n⏳ Waiting for gateway to become READY...")
        if wait_for_gateway_ready():
            print("\n✓ Gateway authorizer configuration fixed successfully")
            return True
        else:
            print("\n✗ Gateway did not reach READY state")
            return False

    except ClientError as e:
        print(f"\n✗ Error updating gateway: {e}")
        return False


# Validate and fix gateway authorizer if needed
validate_and_fix_gateway_authorizer()

---

## パート 2: ヘルパー関数

これらのユーティリティ関数は、トークン管理、API 呼び出し、レスポンス分析のためにチュートリアル全体で使用されます。

In [ ]:
def get_bearer_token() -> str:
    """
    OAuth2 クライアント認証情報フローを使用してベアラートークンを取得します。

    Returns:
        アクセストークン文字列
    """
    # 設定からスコープを取得（利用可能な場合）
    scope = CONFIG.get("client_info", {}).get("scope", "")

    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
    }

    # スコープが設定されている場合は追加
    if scope:
        data["scope"] = scope

    response = requests.post(
        TOKEN_ENDPOINT,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=data,
    )
    response.raise_for_status()
    return response.json()["access_token"]


def decode_token(access_token: str) -> Dict[str, Any]:
    """
    JWT トークンをデコードしてクレームを検査します（検証なし）。

    Args:
        access_token: JWT アクセストークン

    Returns:
        デコードされたトークンペイロードの辞書
    """
    parts = access_token.split(".")
    if len(parts) != 3:
        raise ValueError("無効な JWT トークン形式です")

    # ペイロードをデコード（必要に応じてパディングを追加）
    payload_encoded = parts[1]
    padding = 4 - len(payload_encoded) % 4
    if padding != 4:
        payload_encoded += "=" * padding

    return json.loads(base64.urlsafe_b64decode(payload_encoded))


def make_gateway_request(
    bearer_token: str, tool_name: str, arguments: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Amazon Bedrock AgentCore Gateway に JSON-RPC リクエストを送信します。

    Args:
        bearer_token: OAuth2 アクセストークン
        tool_name: 呼び出すツールの名前
        arguments: ツールの引数

    Returns:
        JSON-RPC レスポンス
    """
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments},
    }

    response = requests.post(
        GATEWAY_URL,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {bearer_token}",
            "Accept": "application/json",
        },
        json=payload,
    )
    response.raise_for_status()
    return response.json()


def analyze_response(result: Dict[str, Any]) -> str:
    """
    Gateway レスポンスを分析して結果を判定します。

    Returns:
        'ALLOWED'、'DENIED'、または 'ERROR'
    """
    # JSON-RPC エラーを確認（ポリシー拒否は特定のメッセージを含むエラーとして返される）
    if "error" in result:
        error_msg = result["error"].get("message", "").lower()
        # ポリシー拒否は 'tool call not allowed'、'access denied' などのエラーを返す
        if any(
            phrase in error_msg
            for phrase in ["not allowed", "denied", "forbidden", "unauthorized action"]
        ):
            return "DENIED"
        return "ERROR"

    if "result" in result:
        # 結果がエラーを示しているか確認（一部の拒否はこの方法で返される）
        if result["result"].get("isError", False):
            content = result["result"].get("content", [])
            if content:
                text = (
                    content[0].get("text", "").lower()
                    if isinstance(content[0], dict)
                    else str(content[0]).lower()
                )
                if any(
                    phrase in text for phrase in ["not allowed", "denied", "forbidden"]
                ):
                    return "DENIED"
            return "DENIED"
        return "ALLOWED"

    return "UNKNOWN"


def display_test_result(expected: str, actual: str, description: str) -> bool:
    """
    テスト結果をフォーマットして表示します。

    Returns:
        テストが成功した場合は True、失敗した場合は False
    """
    passed = expected == actual
    status = "✓ 成功" if passed else "✗ 失敗"
    print(f"\n{status}: {description}")
    print(f"   期待値: {expected}")
    print(f"   実際の値: {actual}")
    return passed


print("✓ ヘルパー関数を定義しました")

---

## パート 3: Amazon Cognito Lambda トリガーの設定

JWT トークンにカスタムクレームを追加するには、Amazon Cognito で Pre Token Generation AWS Lambda トリガーを設定する必要があります。

### 重要な注意事項

- M2M（machine-to-machine）クライアント認証情報フローでは、AWS Lambda トリガーバージョン **V3_0** を使用する**必要があります**
- V3_0 には Amazon Cognito **Essentials** または **Plus** ティアが必要です
- AWS Lambda 関数は `department_name`、`groups` などのカスタムクレームを JWT トークンに追加します
- これらのクレームは Cedar ポリシーで**プリンシパルタグ**になります

In [ ]:
def create_lambda_function(
    claims: Dict[str, Any], function_name: Optional[str] = None
) -> str:
    """
    Pre-token generation トリガー用の Lambda 関数を作成または更新します。

    Args:
        claims: トークンに追加するクレームの辞書
        function_name: オプションのカスタム関数名

    Returns:
        Lambda 関数 ARN
    """
    if function_name is None:
        function_name = f"cognito-custom-claims-{USER_POOL_ID}"

    print(f"\nLambda 関数を設定中: {function_name}")
    print("=" * 70)

    # 指定されたクレームで Lambda コードを生成
    claims_json = json.dumps(claims, indent=12)

    lambda_code = f'''
import json

def lambda_handler(event, context):
    """
    Cognito 用の Pre-token generation V3 Lambda トリガー。
    client_credentials を含むすべてのフローの JWT トークンにカスタムクレームを追加します。
    """
    print(f"イベント: {{json.dumps(event)}}")
    print(f"トリガーソース: {{event.get('triggerSource', 'unknown')}}")
    
    # トークンにカスタムクレームを追加
    event['response'] = {{
        'claimsAndScopeOverrideDetails': {{
            'accessTokenGeneration': {{
                'claimsToAddOrOverride': {claims_json}
            }},
            'idTokenGeneration': {{
                'claimsToAddOrOverride': {claims_json}
            }}
        }}
    }}
    
    print(f"変更されたイベント: {{json.dumps(event)}}")
    return event
'''

    # デプロイパッケージを作成
    with tempfile.NamedTemporaryFile(suffix=".zip", delete=False) as tmp_file:
        zip_path = tmp_file.name
        with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
            zipf.writestr("lambda_function.py", lambda_code)

    try:
        with open(zip_path, "rb") as f:
            zip_content = f.read()

        # 既存の関数を更新を試みる
        try:
            lambda_client.update_function_code(
                FunctionName=function_name, ZipFile=zip_content
            )
            print("✓ Lambda 関数コードを更新しました")
            response = lambda_client.get_function(FunctionName=function_name)
            return response["Configuration"]["FunctionArn"]

        except lambda_client.exceptions.ResourceNotFoundException:
            # IAM ロールで新しい関数を作成
            role_name = f"{function_name}-role"
            role_arn = f"arn:aws:iam::{ACCOUNT_ID}:role/{role_name}"

            # 必要に応じて IAM ロールを作成
            try:
                iam_client.create_role(
                    RoleName=role_name,
                    AssumeRolePolicyDocument=json.dumps(
                        {
                            "Version": "2012-10-17",
                            "Statement": [
                                {
                                    "Effect": "Allow",
                                    "Principal": {"Service": "lambda.amazonaws.com"},
                                    "Action": "sts:AssumeRole",
                                }
                            ],
                        }
                    ),
                )
                iam_client.attach_role_policy(
                    RoleName=role_name,
                    PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
                )
                print(f"✓ IAM ロールを作成しました: {role_name}")
                print("  IAM ロールの伝播を待機中...")
                time.sleep(10)
            except iam_client.exceptions.EntityAlreadyExistsException:
                print(f"  IAM ロールは既に存在します: {role_name}")

            response = lambda_client.create_function(
                FunctionName=function_name,
                Runtime="python3.12",
                Role=role_arn,
                Handler="lambda_function.lambda_handler",
                Code={"ZipFile": zip_content},
                Timeout=30,
                MemorySize=128,
            )
            print("✓ Lambda 関数を作成しました")
            return response["FunctionArn"]
    finally:
        os.remove(zip_path)


print("✓ Lambda 作成関数を定義しました")

In [ ]:
def configure_cognito_trigger(lambda_arn: str) -> None:
    """
    Cognito User Pool に Lambda トリガー V3_0 を設定します。

    Args:
        lambda_arn: Lambda 関数 ARN
    """
    print("\nCognito User Pool トリガーを設定中")
    print("=" * 70)

    # V3_0 トリガーで User Pool を更新（M2M に必須）
    cognito_client.update_user_pool(
        UserPoolId=USER_POOL_ID,
        LambdaConfig={
            "PreTokenGenerationConfig": {
                "LambdaVersion": "V3_0",
                "LambdaArn": lambda_arn,
            }
        },
    )
    print("✓ User Pool トリガーを設定しました（V3_0）")
    print(f"  User Pool ID: {USER_POOL_ID}")
    print(f"  Lambda ARN: {lambda_arn}")

    # Cognito 用の Lambda 権限を追加
    try:
        lambda_client.add_permission(
            FunctionName=lambda_arn,
            StatementId=f"CognitoInvoke-{USER_POOL_ID}",
            Action="lambda:InvokeFunction",
            Principal="cognito-idp.amazonaws.com",
            SourceArn=f"arn:aws:cognito-idp:{REGION}:{ACCOUNT_ID}:userpool/{USER_POOL_ID}",
        )
        print("✓ Cognito 用の Lambda 権限を追加しました")
    except lambda_client.exceptions.ResourceConflictException:
        print("  Lambda 権限は既に存在します")

    print("\n⚠️  重要: V3_0 トリガーには Cognito Essentials または Plus ティアが必要です")


print("✓ Cognito トリガー関数を定義しました")

---

## パート 4: Policy Engine 関数

これらの関数は、Policy for Amazon Bedrock AgentCore サービスと連携して Cedar ポリシーを作成および管理します。

### JWT クレームの Cedar ポリシー構文

JWT クレームは Cedar ポリシーで**プリンシパルタグ**経由でアクセスされます：

| パターン | Cedar 構文 |
|---------|-------------|
| クレームの存在確認 | `principal.hasTag("claim_name")` |
| 完全一致 | `principal.getTag("claim_name") == "value"` |
| パターンマッチ | `principal.getTag("claim_name") like "*value*"` |
| 入力検証 | `context.input.field <= value` |

In [ ]:
def create_policy_engine(name: str) -> Optional[str]:
    """
    新しい Policy Engine を作成します。

    Args:
        name: Policy Engine の名前

    Returns:
        成功した場合は Policy Engine ID、それ以外は None
    """
    print(f"\nPolicy Engine を作成中: {name}")
    print("=" * 70)

    try:
        import uuid

        response = policy_client.create_policy_engine(
            name=name,
            description=f"Policy engine created at {time.strftime('%Y-%m-%d %H:%M:%S')}",
            clientToken=str(uuid.uuid4()),
        )

        policy_engine_id = response["policyEngineId"]
        print("✓ Policy Engine を作成しました")
        print(f"  Policy Engine ID: {policy_engine_id}")

        return policy_engine_id

    except ClientError as e:
        print(f"✗ Policy Engine の作成エラー: {e}")
        return None


def get_policy_engine(policy_engine_id: str) -> Optional[Dict[str, Any]]:
    """
    Policy Engine の詳細を取得します。
    """
    try:
        return policy_client.get_policy_engine(policyEngineId=policy_engine_id)
    except ClientError:
        return None


def wait_for_policy_engine_active(policy_engine_id: str, timeout: int = 300) -> bool:
    """
    Policy Engine が ACTIVE 状態になるまで待機します。

    Args:
        policy_engine_id: Policy Engine ID
        timeout: 最大待機時間（秒）

    Returns:
        ACTIVE の場合は True、タイムアウトまたは失敗の場合は False
    """
    print("\nPolicy Engine が ACTIVE になるまで待機中...")
    start_time = time.time()

    while time.time() - start_time < timeout:
        engine = get_policy_engine(policy_engine_id)
        if not engine:
            time.sleep(5)
            continue

        status = engine.get("status")
        print(f"  ステータス: {status}")

        if status == "ACTIVE":
            print("✓ Policy Engine が ACTIVE になりました")
            return True

        if status in ["CREATE_FAILED", "UPDATE_FAILED", "DELETE_FAILED"]:
            print(f"✗ Policy Engine が失敗しました: {engine.get('statusReason', '不明')}")
            return False

        time.sleep(5)

    print("✗ Policy Engine の待機がタイムアウトしました")
    return False


print("✓ Policy Engine 関数を定義しました")

In [ ]:
def create_cedar_policy(
    policy_name: str, cedar_statement: str, description: str = ""
) -> Optional[str]:
    """
    Policy Engine に Cedar ポリシーを作成します。

    Args:
        policy_name: ポリシーの一意の名前
        cedar_statement: Cedar ポリシーステートメント
        description: ポリシーの説明

    Returns:
        成功した場合はポリシー ID、失敗した場合は None
    """
    print(f"\nCedar ポリシーを作成中: {policy_name}")
    print("=" * 70)
    print("\nCedar ステートメント:")
    print("-" * 60)
    print(cedar_statement)
    print("-" * 60)

    try:
        response = policy_client.create_policy(
            policyEngineId=POLICY_ENGINE_ID,
            name=policy_name,
            description=description or f"Policy: {policy_name}",
            definition={"cedar": {"statement": cedar_statement}},
        )

        policy_id = response["policyId"]
        policy_status = response["status"]

        print("\n✓ ポリシーを正常に作成しました")
        print(f"  ポリシー ID: {policy_id}")
        print(f"  ステータス: {policy_status}")

        return policy_id

    except ClientError as e:
        error_code = e.response["Error"]["Code"]
        error_msg = e.response["Error"]["Message"]
        print(f"\n✗ ポリシーの作成エラー: {error_code}")
        print(f"  {error_msg}")
        return None


def get_policy(policy_id: str) -> Optional[Dict[str, Any]]:
    """
    ステータスを含むポリシー詳細を取得します。

    Args:
        policy_id: 取得するポリシー ID

    Returns:
        ポリシー詳細の辞書、見つからない場合は None
    """
    try:
        return policy_client.get_policy(
            policyEngineId=POLICY_ENGINE_ID, policyId=policy_id
        )
    except ClientError:
        return None


def wait_for_policy_active(policy_id: str, timeout: int = 60) -> bool:
    """
    ポリシーが ACTIVE ステータスになるまで待機します。

    Args:
        policy_id: 確認するポリシー ID
        timeout: 最大待機時間（秒）

    Returns:
        ACTIVE の場合は True、それ以外は False
    """
    start_time = time.time()

    while time.time() - start_time < timeout:
        policy = get_policy(policy_id)
        if not policy:
            print(f"  ⚠️  ポリシーが見つかりません: {policy_id}")
            return False

        status = policy.get("status")
        print(f"  ポリシーステータス: {status}")

        if status == "ACTIVE":
            return True

        if status in ["CREATE_FAILED", "UPDATE_FAILED"]:
            print(f"  ✗ ポリシーが失敗しました: {policy.get('statusReason', '不明')}")
            return False

        time.sleep(3)

    print("  ✗ ポリシーが ACTIVE になるまでのタイムアウト")
    return False


def delete_policy(policy_id: str) -> bool:
    """
    Policy Engine からポリシーを削除します。
    """
    try:
        policy_client.delete_policy(policyEngineId=POLICY_ENGINE_ID, policyId=policy_id)
        print(f"✓ ポリシーを削除しました: {policy_id}")
        return True
    except ClientError as e:
        print(f"⚠️  ポリシー {policy_id} を削除できませんでした: {e}")
        return False


def list_policies() -> List[Dict[str, Any]]:
    """
    Policy Engine 内のすべてのポリシーを一覧表示します。
    """
    try:
        response = policy_client.list_policies(policyEngineId=POLICY_ENGINE_ID)
        return response.get("policies", [])
    except ClientError:
        return []


print("✓ Cedar ポリシー関数を定義しました")

### ステップ 4.1: Policy Engine の存在確認

Policy Engine が存在するか確認し、必要に応じて作成します。

In [ ]:
def ensure_policy_engine() -> str:
    """
    Ensure a policy engine exists and is active.
    Creates one if needed and updates gateway_config.json.

    Returns:
        Policy engine ID
    """
    global POLICY_ENGINE_ID, CONFIG

    print("\nEnsuring Policy Engine Exists")
    print("=" * 70)

    # Check if we already have a policy engine ID
    if POLICY_ENGINE_ID:
        engine = get_policy_engine(POLICY_ENGINE_ID)
        if engine and engine.get("status") == "ACTIVE":
            print(f"✓ Using existing policy engine: {POLICY_ENGINE_ID}")
            return POLICY_ENGINE_ID

    # List existing policy engines
    try:
        response = policy_client.list_policy_engines()
        engines = response.get("policyEngines", [])

        for engine in engines:
            if engine.get("status") == "ACTIVE":
                POLICY_ENGINE_ID = engine["policyEngineId"]
                print(f"✓ Found existing ACTIVE policy engine: {POLICY_ENGINE_ID}")
                break
    except ClientError:
        pass

    # Create new policy engine if needed
    if not POLICY_ENGINE_ID:
        engine_name = f"PolicyEngine_{int(time.time())}"
        POLICY_ENGINE_ID = create_policy_engine(engine_name)

        if not POLICY_ENGINE_ID:
            raise RuntimeError("Failed to create policy engine")

        if not wait_for_policy_engine_active(POLICY_ENGINE_ID):
            raise RuntimeError("Policy engine did not become ACTIVE")

    # Save to gateway_config.json
    CONFIG["policy_engine_id"] = POLICY_ENGINE_ID
    with open("gateway_config.json", "w") as f:
        json.dump(CONFIG, f, indent=2)
    print("✓ Saved policy_engine_id to gateway_config.json")

    return POLICY_ENGINE_ID


# Ensure policy engine exists
POLICY_ENGINE_ID = ensure_policy_engine()

### ステップ 4.2: Policy Engine を Gateway に接続

ポリシーを適用するには、Policy Engine を Gateway に接続する必要があります。このステップでは、Gateway に Policy Engine が既に設定されているか確認し、設定されていない場合は接続します。

In [ ]:
# Note: gateway_control_client, get_gateway_details, and wait_for_gateway_ready
# are already defined in Step 1.3


def attach_policy_engine_to_gateway(mode: str = "ENFORCE") -> bool:
    """
    Attach the Policy Engine to the Gateway if not already attached.

    Args:
        mode: Policy engine mode ('LOG_ONLY' or 'ENFORCE')

    Returns:
        True if successful or already attached, False otherwise
    """
    print("\nAttaching Policy Engine to Gateway")
    print("=" * 70)

    # Get current gateway configuration
    gateway_config = get_gateway_details()

    # Check if policy engine is already attached
    existing_pe = gateway_config.get("policyEngineConfiguration", {})
    if existing_pe.get("arn"):
        print(f"✓ Policy Engine already attached: {existing_pe.get('arn')}")
        print(f"  Mode: {existing_pe.get('mode', 'N/A')}")
        return True

    # Get policy engine ARN
    engine = get_policy_engine(POLICY_ENGINE_ID)
    if not engine:
        print("✗ Could not get policy engine details")
        return False

    policy_engine_arn = engine.get("policyEngineArn")
    print(f"  Policy Engine ARN: {policy_engine_arn}")
    print(f"  Mode: {mode}")

    try:
        # Use boto3 client to update gateway
        gateway_control_client.update_gateway(
            gatewayIdentifier=GATEWAY_ID,
            name=gateway_config.get("name"),
            roleArn=gateway_config.get("roleArn"),
            protocolType=gateway_config.get("protocolType", "MCP"),
            authorizerType=gateway_config.get("authorizerType", "CUSTOM_JWT"),
            policyEngineConfiguration={"arn": policy_engine_arn, "mode": mode},
        )

        print("✓ Gateway update request accepted")
        print("\n⏳ Waiting for gateway to become READY...")

        if wait_for_gateway_ready():
            print("✓ Policy Engine attached successfully")
            return True
        else:
            print("✗ Gateway did not reach READY state")
            return False

    except ClientError as e:
        print(f"✗ Error updating gateway: {e}")
        return False


# Attach policy engine to gateway
attach_policy_engine_to_gateway(mode="ENFORCE")

---

## パート 5: テストシナリオ 1 - 部門ベースのアクセス制御

このシナリオでは、**finance** 部門のユーザーからのリクエストのみを許可するポリシーを作成します。

### Cedar ポリシーパターン

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
};
```

### ステップ 5.0: 既存ポリシーのクリーンアップ（オプション）

新しいポリシーを作成する前に、クリーンなテスト環境を確保するために既存のポリシーを削除することをお勧めします。これにより、古いポリシーと新しいポリシー間の競合を防ぎます。

In [ ]:
def cleanup_existing_policies(require_confirmation: bool = True) -> int:
    """
    Delete all existing policies in the policy engine.

    Args:
        require_confirmation: If True, asks user for confirmation before deleting

    Returns:
        Number of policies deleted
    """
    print("\n🧹 Checking for existing policies...")
    print("=" * 70)

    policies = list_policies()

    if not policies:
        print("✓ No existing policies found. Ready to proceed.")
        return 0

    print(f"\n⚠️  Found {len(policies)} existing policy/policies:")
    for p in policies:
        print(
            f"   - {p.get('name', 'unnamed')} (ID: {p.get('policyId')}, Status: {p.get('status')})"
        )

    if require_confirmation:
        print("\n" + "-" * 70)
        confirm = (
            input("Do you want to DELETE all existing policies? (yes/no): ")
            .strip()
            .lower()
        )
        if confirm != "yes":
            print("\n⏭️  Skipping cleanup. Existing policies will remain.")
            print("   Note: This may cause unexpected policy evaluation results.")
            return 0

    print("\n🗑️  Deleting existing policies...")
    deleted_count = 0
    for p in policies:
        policy_id = p.get("policyId")
        if policy_id and delete_policy(policy_id):
            deleted_count += 1

    print(f"\n✓ Deleted {deleted_count}/{len(policies)} policies")
    return deleted_count


# Track created policies for cleanup at the end
CREATED_POLICIES = []

# Clean up existing policies before starting tests
cleanup_existing_policies(require_confirmation=True)

### ステップ 5.1: Finance 部門クレームで Lambda を設定

In [ ]:
print("=" * 70)
print("テストシナリオ 1: 部門ベースのアクセス制御")
print("=" * 70)

# Configure Lambda with department_name = "finance"
claims_finance = {
    "department_name": "finance",
    "employee_level": "senior",
    "cost_center": "CC-1001",
}

lambda_arn = create_lambda_function(claims_finance)
configure_cognito_trigger(lambda_arn)

print("\n✓ Lambda configured with claims:")
print(json.dumps(claims_finance, indent=2))

### ステップ 5.2: トークンにカスタムクレームが含まれていることを確認

In [ ]:
print("\nVerifying Token Claims")
print("=" * 70)

token = get_bearer_token()
claims = decode_token(token)

print("\nToken Claims (relevant):")
print(f"  department_name: {claims.get('department_name', 'NOT PRESENT')}")
print(f"  employee_level: {claims.get('employee_level', 'NOT PRESENT')}")
print(f"  cost_center: {claims.get('cost_center', 'NOT PRESENT')}")
print(f"  client_id: {claims.get('client_id', 'NOT PRESENT')}")

if claims.get("department_name") == "finance":
    print("\n✓ Custom claims verified in token")
else:
    print(
        "\n⚠️  Custom claims not found - Lambda trigger may not be configured correctly"
    )

### ステップ 5.3: 部門検証用の Cedar ポリシーを作成

In [ ]:
policy_name = f"dept_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
}};'''

print(f"Cedar ステートメント:\n{cedar_statement}")

policy_id = create_cedar_policy(
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow requests only from finance department",
)

if policy_id:
    CREATED_POLICIES.append(policy_id)

    # Wait for policy to become ACTIVE (required before testing)
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_id):
        print("✓ Policy is ACTIVE and ready for testing")
    else:
        print("\n⚠️  Policy did not become ACTIVE. Tests may fail.")
        print("   Check the policy status in the AWS Console.")
else:
    print("\n✗ Failed to create policy. Cannot proceed with tests.")

### ステップ 5.4: Finance 部門でテスト（期待: 許可）

In [ ]:
print("\n" + "=" * 70)
print("テスト 1.1: department_name='finance' でのリクエスト")
print("=" * 70)

token = get_bearer_token()
result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-finance"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "Finance department should be ALLOWED")

### ステップ 5.5: Engineering 部門でテスト（期待: 拒否）

In [ ]:
print("\n" + "=" * 70)
print("テスト 1.2: department_name='engineering' でのリクエスト")
print("=" * 70)

# Update Lambda with different department
claims_engineering = {
    "department_name": "engineering",
    "employee_level": "senior",
    "cost_center": "CC-2001",
}

lambda_arn = create_lambda_function(claims_engineering)
print("\n✓ Lambda updated with department_name='engineering'")

# Wait for Lambda changes to propagate
print("\n⏳ Waiting for Lambda changes to propagate...")
time.sleep(5)

# Get new token and test
token = get_bearer_token()
claims = decode_token(token)
print(f"\nToken department_name: {claims.get('department_name')}")

result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-engineering"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "Engineering department should be DENIED")

---

## パート 6: テストシナリオ 2 - グループベースのアクセス制御

このシナリオでは、**admins** グループに属するユーザーからのリクエストのみを許可するポリシーを作成します。

### Cedar ポリシーパターン

groups はトークン内で文字列としてシリアライズされるため、パターンマッチングに `like` 演算子を使用します：

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("テストシナリオ 2: グループベースのアクセス制御")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(pid)
CREATED_POLICIES.clear()

### ステップ 6.1: グループクレームで Lambda を設定

In [ ]:
# Configure Lambda with groups containing "admins"
claims_with_admins = {
    "groups": ["admins", "developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(claims_with_admins)
configure_cognito_trigger(lambda_arn)

print("\n✓ Lambda configured with groups:")
print(f"   groups: {claims_with_admins['groups']}")

### ステップ 6.2: トークンにグループクレームが含まれていることを確認

In [ ]:
print("\nVerifying Token Claims")
print("=" * 70)

token = get_bearer_token()
claims = decode_token(token)

print("\nToken Claims (relevant):")
print(f"  groups: {claims.get('groups', 'NOT PRESENT')}")
print(f"  department_name: {claims.get('department_name', 'NOT PRESENT')}")

### ステップ 6.3: グループ検証用の Cedar ポリシーを作成

In [ ]:
policy_name = f"groups_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
}};'''

print(f"Cedar ステートメント:\n{cedar_statement}")

policy_id = create_cedar_policy(
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow requests only from users in admins group",
)

if policy_id:
    CREATED_POLICIES.append(policy_id)

    # Wait for policy to become ACTIVE (required before testing)
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_id):
        print("✓ Policy is ACTIVE and ready for testing")
    else:
        print("\n⚠️  Policy did not become ACTIVE. Tests may fail.")
else:
    print("\n✗ Failed to create policy. Cannot proceed with tests.")

### ステップ 6.4: Admins グループでテスト（期待: 許可）

In [ ]:
print("\n" + "=" * 70)
print("テスト 2.1: groups=['admins', 'developers', 'team-alpha'] でのリクエスト")
print("=" * 70)

token = get_bearer_token()
result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-admins"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "User with 'admins' group should be ALLOWED")

### ステップ 6.5: Admins グループなしでテスト（期待: 拒否）

In [ ]:
print("\n" + "=" * 70)
print("テスト 2.2: groups=['developers', 'team-alpha'] でのリクエスト")
print("=" * 70)

# Update Lambda without admins group
claims_no_admins = {
    "groups": ["developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(claims_no_admins)
print("\n✓ Lambda updated with groups (no admins):")

# Wait for Lambda changes to propagate
print("\n⏳ Waiting for Lambda changes to propagate...")
time.sleep(5)
print(f"   groups: {claims_no_admins['groups']}")

# Get new token and test
token = get_bearer_token()
claims = decode_token(token)
print(f"\nToken groups: {claims.get('groups')}")

result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-no-admins"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "User without 'admins' group should be DENIED")

---

## パート 7: テストシナリオ 3 - プリンシパル ID ベースのアクセス制御

このシナリオでは、特定のプリンシパル（クライアント認証情報フローの JWT トークンの `sub` クレームで識別）からのリクエストのみを許可するポリシーを作成します。

### Cedar ポリシーパターン

Policy for Amazon Bedrock AgentCore では、プリンシパルには評価可能な単一の属性 `id` があります。クライアント認証情報フローでは、これは JWT トークンの `sub` クレーム（`client_id` と等しい）に対応します：

```cedar
permit(principal, action, resource)
when {
    principal.id == "your-client-id"
};
```

**注意**: `principal.hasTag()` と `principal.getTag()` 経由でアクセスするカスタムクレームとは異なり、プリンシパル ID は `principal.id` 経由で直接アクセスします。

In [ ]:
# Clean up previous policy
print("=" * 70)
print("テストシナリオ 3: プリンシパル ID ベースのアクセス制御")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(pid)
CREATED_POLICIES.clear()

### ステップ 7.1: クライアント ID 検証用の Cedar ポリシーを作成

In [ ]:
policy_name = f"principal_id_policy_{int(time.time())}"

# In Client Credentials Flow, principal.id equals the 'sub' claim (which is the client_id)
cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.id == "{CLIENT_ID}"
}};'''

print(f"Cedar ステートメント:\n{cedar_statement}")

policy_id = create_cedar_policy(
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description=f"Allow requests only from principal.id: {CLIENT_ID}",
)

if policy_id:
    CREATED_POLICIES.append(policy_id)

    # Wait for policy to become ACTIVE
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_id):
        print("✓ Policy is ACTIVE and ready for testing")
    else:
        print("\n⚠️  Policy did not become ACTIVE. Tests may fail.")

### ステップ 7.2: 一致するプリンシパル ID でテスト（期待: 許可）

In [ ]:
print("\n" + "=" * 70)
print(f"テスト 3.1: principal.id='{CLIENT_ID}' でのリクエスト")
print("=" * 70)

token = get_bearer_token()
claims = decode_token(token)
# In Client Credentials Flow, 'sub' claim equals client_id and maps to principal.id
print(f"\nToken 'sub' claim (principal.id): {claims.get('sub')}")
print(f"Token 'client_id' claim: {claims.get('client_id')}")

result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-principal-id"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "Matching principal.id should be ALLOWED")

print("\n💡 Note: To test DENY scenario, you would need a different")
print("   Amazon Cognito app client with a different client_id (sub)")

---

## パート 8: 高度なパターン

### 複数条件の組み合わせ

より複雑なアクセス制御シナリオのために、単一のポリシーで複数の条件を組み合わせることができます。

In [ ]:
# Example: Combined policy (department AND amount limit)
print("\nAdvanced Pattern: Combined Conditions")
print("=" * 70)

combined_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance" &&
    context.input.amount <= 1000
}};'''

print("Cedar Policy with Combined Conditions:")
print("-" * 60)
print(combined_cedar)
print("-" * 60)
print("\nThis policy allows requests when:")
print("  ✓ User is in finance department")
print("  ✓ AND refund amount is <= $1000")

### `like` 演算子によるパターンマッチング

`like` 演算子は柔軟なマッチングのためにワイルドカードをサポートします：

| パターン | マッチ対象 |
|---------|--------|
| `"*admin*"` | 任意の位置に "admin" を含む |
| `"admin*"` | "admin" で始まる |
| `"*admin"` | "admin" で終わる |
| `"team-*"` | "team-" で始まる |

In [ ]:
# Example: Pattern matching for team-based access
print("\nAdvanced Pattern: Team-Based Access with Wildcards")
print("=" * 70)

team_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*team-finance*"
}};'''

print("Cedar Policy with Pattern Matching:")
print("-" * 60)
print(team_cedar)
print("-" * 60)
print("\nThis policy allows requests when:")
print("  ✓ User's groups contain 'team-finance'")
print("  ✓ Matches: ['team-finance', 'developers']")
print("  ✓ Matches: ['admins', 'team-finance-leads']")

---

## パート 9: ベストプラクティス

### ポリシー設計のベストプラクティス

1. **具体的なアクションを使用** - ワイルドカードよりも具体的なツールアクションをターゲットにする
2. **常にクレームの存在を確認** - エラーを避けるため `getTag()` の前に `hasTag()` を使用
3. **パターンマッチングは慎重に** - `like "*value*"` は意図しない文字列にマッチする可能性
4. **許可と拒否の両方をテスト** - ポリシーが両方向で機能することを確認
5. **ポリシーを文書化** - 説明的な名前と説明を使用

### Amazon Cognito 設定のベストプラクティス

1. **V3_0 トリガーを使用** - M2M クライアント認証情報フローに必須
2. **Essentials ティアにアップグレード** - V3_0 には Amazon Cognito Essentials または Plus が必要
3. **トークンクレームをテスト** - ポリシー作成前にクレームがトークンに含まれていることを常に確認
4. **配列は慎重に扱う** - 配列は JWT クレームで文字列としてシリアライズされる

### 避けるべき一般的な落とし穴

- V1_0 または V2_0 トリガーを M2M フローで使用（クレームが追加されない）
- `getTag()` の前に `hasTag()` を確認し忘れる
- 配列に対してパターンマッチ（`like`）が必要な場合に完全一致（`==`）を使用
- テスト前にポリシーが ACTIVE になるのを待たない
- 適切なクリーンアップ戦略なしにポリシーを作成

### 必要な AWS IAM 権限

#### ポリシー管理用

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "policy-registry:CreatePolicyEngine",
        "policy-registry:GetPolicyEngine",
        "policy-registry:ListPolicyEngines",
        "policy-registry:CreatePolicy",
        "policy-registry:DeletePolicy",
        "policy-registry:ListPolicies"
      ],
      "Resource": "*"
    }
  ]
}
```

#### Amazon Cognito AWS Lambda トリガー用

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "cognito-idp:UpdateUserPool",
        "lambda:CreateFunction",
        "lambda:UpdateFunctionCode",
        "lambda:AddPermission",
        "iam:CreateRole",
        "iam:AttachRolePolicy",
        "iam:PassRole"
      ],
      "Resource": "*"
    }
  ]
}
```

---

## パート 10: クリーンアップ

このチュートリアルで作成したすべてのポリシーを削除します。

In [ ]:
print("=" * 70)
print("CLEANUP")
print("=" * 70)

print(f"\nDeleting {len(CREATED_POLICIES)} policies...")
for pid in CREATED_POLICIES:
    delete_policy(pid)

CREATED_POLICIES.clear()
print("\n✓ Cleanup complete")

### オプション: すべてのポリシーを削除

Policy Engine 内のすべてのポリシーをクリーンアップするために使用します（注意して使用してください）。

In [ ]:
# Uncomment to delete ALL policies in the policy engine
# WARNING: This will delete all policies, not just those created in this tutorial

# print("Deleting ALL policies...")
# policies = list_policies()
# for policy in policies:
#     policy_id = policy.get('policyId')
#     if policy_id:
#         delete_policy(policy_id)
# print("✓ All policies deleted")

---

## まとめ

おめでとうございます！Policy for Amazon Bedrock AgentCore チュートリアルを完了しました。以下を学習しました：

- Amazon Cognito AWS Lambda トリガーを設定して JWT トークンにカスタムクレームを追加する方法
- プリンシパルタグ経由で JWT クレームを検証する Cedar ポリシーの作成方法
- 部門ベースのアクセス制御の実装方法
- パターンマッチングを使用したグループベースのアクセス制御の実装方法
- プリンシパル ID ベースのアクセス制御の実装方法
- 複雑なアクセス制御シナリオのための複数条件の組み合わせ方法

### 主要な Cedar 構文パターン

| クレームタイプ | Cedar 構文 |
|------------|-------------|
| 文字列（完全一致） | `principal.getTag("claim") == "value"` |
| 文字列（含む） | `principal.getTag("claim") like "*value*"` |
| 配列（含む） | `principal.getTag("claim") like "*value*"` |
| 入力検証 | `context.input.field <= value` |

### 次のステップ

1. **本番環境で実装** - これらのパターンを Amazon Bedrock AgentCore デプロイメントに適用
2. **ポリシーをカスタマイズ** - 特定のアクセス制御要件に合わせてポリシーを調整
3. **モニタリングを追加** - ポリシー拒否に対する Amazon CloudWatch アラームを設定
4. **反復と改善** - 実際の使用状況に基づいてポリシーを改善

---

**作成者**: AWS  
**ライセンス**: MIT-0  
**最終更新日**: 2025